# The finite field sum-product problem

In [ ]:
#@title Code found by AlphaEvolve (intersection of an AP and a GP)

"""AlphaEvolve experiment for the finite field sum-product problem."""
import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from collections.abc import Callable, Mapping
from typing import Any, List, Tuple
import scipy.linalg as la
import collections
import copy
import math
import numba
from scipy.optimize import milp, LinearConstraint, Bounds
from itertools import product

njit = numba.njit


@njit
def _get_intersection_positive(L: int, g: int, p: int, required_size: int) -> (np.ndarray, bool):
  """
  Computes intersection of AP={1..L} and GP={g^0..g^{L-1}} mod p.

  Returns a tuple (intersection_array, is_large_enough).
  A two-pass approach is used to handle array allocation efficiently in numba.
  """
  # First pass: count elements to determine required array size.
  count = 0
  current_power = 1
  for _ in range(L):
    if 1 <= current_power <= L:
      count += 1
    current_power = (current_power * g) % p

  if count < required_size:
    # Return empty array and False if not enough elements.
    return (np.empty(0, dtype=np.int64), False)

  # Second pass: fill the pre-allocated array.
  intersection = np.empty(count, dtype=np.int64)
  current_power = 1
  idx = 0
  for _ in range(L):
    if 1 <= current_power <= L:
      intersection[idx] = current_power
      idx += 1
    current_power = (current_power * g) % p

  return (intersection, True)


@njit
def _get_intersection_centered(L: int, g: int, p: int, required_size: int) -> (np.ndarray, bool):
  """
  Computes intersection of AP={-(L-1)//2..L//2} and GP={g^0..g^{L-1}} mod p.

  The AP is centered around 0 and is wrapped modulo p.
  Returns a tuple (intersection_array, is_large_enough).
  """
  start = -(L - 1) // 2
  end = L // 2

  # First pass: count elements to determine required array size.
  count = 0
  current_power = 1
  for _ in range(L):
    # Check if current_power is in [start, end] wrapped mod p.
    # This corresponds to {0..end} U {p+start..p-1}.
    if current_power <= end or current_power >= p + start:
      count += 1
    current_power = (current_power * g) % p

  if count < required_size:
    # Return empty array and False if not enough elements.
    return (np.empty(0, dtype=np.int64), False)

  # Second pass: fill the pre-allocated array.
  intersection = np.empty(count, dtype=np.int64)
  current_power = 1
  idx = 0
  for _ in range(L):
    if current_power <= end or current_power >= p + start:
      intersection[idx] = current_power
      idx += 1
    current_power = (current_power * g) % p

  return (intersection, True)


def _find_min_L_for_g(g: int, p: int, size: int, intersection_func: Callable) -> Tuple[float, Any]:
  """
  For a given generator g, finds the minimum L such that
  | AP intersect GP | >= size, for a given AP construction.
  Uses binary search over L.
  """
  # Establish a search range for L.
  # Lower bound is `size` as intersection can't be larger than L.
  L_low = size
  # Upper bound is based on the heuristic E[|X|] ~ L^2/p = size -> L ~ sqrt(size*p) = p^0.75
  # The factor 2.0 is a safety margin. This is a tighter bound than before,
  # justified by empirical results, to speed up the search.
  L_high = int(2.0 * p**0.75)

  best_L_for_g = float('inf')
  best_intersection_for_g = None

  # Quick check: if the upper bound doesn't work, this g is not promising.
  intersection, success = intersection_func(L_high, g, p, size)
  if not success:
    return float('inf'), None

  # Binary search for the minimal L.
  while L_low <= L_high:
    L_mid = L_low + (L_high - L_low) // 2
    if L_mid == 0:
      L_low = 1
      continue

    intersection, success = intersection_func(L_mid, g, p, size)

    if success:
      # This L works. Store it and try to find a smaller one.
      best_L_for_g = L_mid
      best_intersection_for_g = intersection
      L_high = L_mid - 1
    else:
      # This L is too small. Need to increase it.
      L_low = L_mid + 1

  return best_L_for_g, best_intersection_for_g


def _get_cubic_g_candidates(p: int) -> List[int]:
  """Generates g candidates from cubic roots x^3=c for p=2 mod 3."""
  if p % 3 != 2:
    return []

  # if p=2 mod 3, then gcd(3, p-1)=1, so a unique cube root exists.
  # e = 3^{-1} mod (p-1)
  try:
    e = pow(3, -1, p - 1)
  except ValueError:
    # This should not be reached if p % 3 == 2
    return []

  candidates = []
  B_c = 10  # Bound for c
  for c in range(2, B_c + 1):
    g = pow(c, e, p)
    candidates.append(g)
  return candidates


def _get_quadratic_g_candidates(p: int) -> List[int]:
  """
  Generates g candidates from quadratic extensions for p = 5 mod 8.
  """
  # p must be 1 mod 4, so p is either 1 or 5 mod 8. This handles the 5 mod 8 case.
  if p % 8 != 5:
    return []

  # Find smallest prime d that is a quadratic residue mod p.
  # Since p = 1 (mod 4), by quadratic reciprocity (d/p) = (p/d).
  # We need p to be a quadratic residue mod d.
  primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31]
  d = -1
  for prime_d in primes:
    if p == prime_d: continue
    if pow(p, (prime_d - 1) // 2, prime_d) == 1:
      d = prime_d
      break
  if d == -1:
    return []

  # Simple modular sqrt for p = 5 (mod 8)
  # This formula gives either sqrt(d) or sqrt(-d).
  s_cand = pow(d, (p + 3) // 8, p)

  if pow(s_cand, 2, p) == d:
    s = s_cand
  else:
    # We got sqrt(-d), need to multiply by sqrt(-1).
    # For p=5(mod 8), 2 is a non-residue, so i=2^((p-1)/4) is sqrt(-1).
    i = pow(2, (p - 1) // 4, p)
    s = (s_cand * i) % p

  # Verification - if d was not a QR, this will fail.
  if pow(s, 2, p) != d:
      return []

  # Generate candidates of the form a + b*s
  B_q = 4
  candidates = []
  for a in range(-B_q, B_q + 1):
    for b in range(-B_q, B_q + 1):
      if a == 0 and b == 0:
        continue
      candidates.append((a + b * s) % p)
  return candidates


def search_for_best_construction(p: int) -> np.ndarray:
  """Constructs a set X in F_p of size floor(sqrt(p)).

  The goal is to minimize max(|X+X|, |X*X|).
  """
  size = int(math.sqrt(p))

  # This construction intersects an arithmetic progression (AP) {1, ..., L}
  # with a geometric progression (GP) {1, g, g^2, ..., g^{L-1}}.
  # The sum/product set sizes are bounded by 2L-1. We minimize this bound
  # by finding the smallest L that gives an intersection of at least `size`.
  # This is done by searching over generators `g`. Candidates for `g` are
  # chosen from three families:
  # 1. Rational numbers a/b with small a, b.
  # 2. Elements a+b*sqrt(d) from a quadratic extension, for small a, b, d.
  # 3. Cubic roots of small integers, for primes p where this is simple.
  # For each g, the minimal L that gives an intersection of at least `size`
  # is found via binary search.

  g_candidates = set()

  # 1. Candidates from rational numbers a/b.
  B = 10 # Bound for numerator and denominator of g
  for a in range(1, B + 1):
    for b in range(1, B + 1):
      if math.gcd(a, b) != 1:
        continue
      # g = a * b^{-1} mod p
      g = (a * pow(b, -1, p)) % p
      if g > 1: # g=0, 1 are trivial
        g_candidates.add(g)

  # 2. Candidates from quadratic extensions.
  quadratic_candidates = _get_quadratic_g_candidates(p)
  for g in quadratic_candidates:
    if g > 1:
      g_candidates.add(g)

  # 3. Candidates from cubic roots.
  cubic_candidates = _get_cubic_g_candidates(p)
  for g in cubic_candidates:
    if g > 1:
      g_candidates.add(g)

  min_L = float('inf')
  best_intersection = None

  g_list = sorted(list(g_candidates))

  # Test different arithmetic progressions. The original used a positive AP {1..L}.
  # A centered AP {-(L-1)//2..L//2} might yield a smaller L for some g.
  # We test both and take the best result.
  # njit'd functions must be available in global scope to be passed as argument.
  intersection_funcs = [_get_intersection_positive, _get_intersection_centered]

  for intersection_func in intersection_funcs:
    for g in g_list:
      L, intersection = _find_min_L_for_g(g, p, size, intersection_func)

      if L < min_L:
        min_L = L
        if intersection is not None:
          best_intersection = np.sort(intersection)

  if best_intersection is not None:
    # We found a good construction. Take the first `size` elements.
    x = best_intersection[:size].astype(np.int64)
  else:
    # Fallback if no g worked. This is unlikely but a necessary safeguard.
    logging.warning(f"AP-GP intersection failed for all candidates, "
                    f"falling back to AP.")
    x = np.arange(size, dtype=np.int64)

  return x

In [ ]:
#@title A second code evolved by AlphaEvolve, using Gaussian integers

"""AlphaEvolve experiment for the finite field sum-product problem."""
import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from collections.abc import Callable, Mapping
from typing import Any, List, Tuple
import scipy.linalg as la
import collections
import copy
import math
import numba
from scipy.optimize import milp, LinearConstraint, Bounds
from itertools import product

njit = numba.njit


@numba.njit
def _lpf_sieve(limit: int) -> np.ndarray:
  """Computes the largest prime factor for all numbers up to limit."""
  lpf = np.zeros(limit + 1, dtype=np.int64)
  for i in range(2, limit + 1):
    if lpf[i] == 0:  # i is prime
      for j in range(i, limit + 1, i):
        lpf[j] = i
  if limit >= 0:
    lpf[0] = 1  # By convention for norm=0
  if limit >= 1:
    lpf[1] = 1
  return lpf


def search_for_best_construction(p: int) -> np.ndarray:
  """Constructs a set X in F_p of size floor(sqrt(p)).

  The goal is to minimize max(|X+X|, |X*X|).
  This implementation uses a construction based on Gaussian integers mod p.
  The set X is constructed from elements a+bi where the norm a^2+b^2 is
  chosen to have small prime factors. This gives the product set a high
  degree of algebraic structure. The additive structure is encouraged by
  selecting points (a,b) with small max(|a|,|b|) as a tie-breaker.
  """
  # The size of the set is fixed to floor(sqrt(p)).
  k = int(math.sqrt(p))

  # Find i such that i^2 = -1 (mod p).
  # This is possible because p is congruent to 1 mod 4.
  # We use Tonelli-Shanks for the simple case p = 1 (mod 4).
  # Find a quadratic non-residue `n` by trial.
  n = 2
  while pow(n, (p - 1) // 2, p) != p - 1:
    n += 1
  # i = n^((p-1)/4) mod p
  i = pow(n, (p - 1) // 4, p)

  # Construct a set of pairs (a, b) corresponding to the elements a+bi.
  # We search in a circular region, which is more natural for a norm-based
  # selection criterion. A larger radius ensures we find enough unique
  # points with good properties (e.g., small LPF of norm).
  # A larger search space offers more high-quality candidate points (those
  # with smaller largest prime factors of norm). The factor is increased
  # from 3.0 to 3.2 to further expand the search, which has been found to yield
  # better sets, at a small cost in computation time.
  radius_search_factor = 3.2
  radius_est = int(radius_search_factor * math.sqrt(k)) + 5

  max_norm = radius_est**2
  lpf_table = _lpf_sieve(max_norm)

  points = []
  for a in range(-radius_est, radius_est + 1):
    b_limit = int(math.sqrt(max(0, radius_est**2 - a**2)))
    for b in range(-b_limit, b_limit + 1):
      norm = a * a + b * b
      lpf = lpf_table[norm]
      # The sorting key prioritizes small LPF of norm, then small norm.
      # As a tie-breaker, we prefer points inside a smaller L-infinity ball
      # (a square), i.e., with a smaller max(|a|, |b|). This should improve
      # the additive structure of the set.
      points.append((lpf, norm, max(abs(a), abs(b)), a, b))

  points.sort()

  # Create the set X from the best points, ensuring k unique elements.
  x_set = set()
  for _, _, _, a, b in points:
    if len(x_set) == k:
      break
    val = (a + b * i) % p
    x_set.add(val)

  return np.array(list(x_set), dtype=np.int64)

In [ ]:
#@title Evaluation code

@njit
def _calculate_sum_prod_numba(
    construction: np.ndarray, p: int
) -> tuple[int, int]:
  """A Numba-jitted function to rapidly calculate sum and product set sizes."""
  n = construction.shape[0]

  # Calculate sum set X+X
  sum_set = {np.int64(0)}  # Initialize with a dummy value
  sum_set.clear()  # Numba needs the type to be inferred
  for i in range(n):
    for j in range(n):
      sum_val = (construction[i] + construction[j]) % p
      sum_set.add(sum_val)

  # Calculate product set X*X
  prod_set = {np.int64(0)}
  prod_set.clear()
  for i in range(n):
    for j in range(n):
      prod_val = (construction[i] * construction[j]) % p
      prod_set.add(prod_val)

  return len(sum_set), len(prod_set)


def calculate_score(construction: np.ndarray, p: int) -> float:
  """Calculates the score for a given sum-product construction."""
  target_size = int(math.sqrt(p))

  # --- Basic Validity Checks (remain in pure Python) ---
  if not isinstance(construction, np.ndarray) or construction.ndim != 1:
    logging.warning('Invalid construction: not a 1D numpy array.')
    return np.inf

  # Ensure all elements are within the field F_p.
  if np.any(construction < 0) or np.any(construction >= p):
    logging.warning('Invalid construction: elements are out of F_p bounds.')
    return np.inf

  # Remove duplicates to get the actual set.
  # Note: Numba doesn't have np.unique, so we do it here.
  if np.unique(construction).shape[0] != construction.shape[0]:
    logging.warning('Invalid construction: contains duplicate elements.')
    return np.inf

  # Check if the construction has the correct size.
  if construction.shape[0] != target_size:
    logging.warning(
        f'Invalid construction: size is {construction.shape[0]},'
        f' expected {target_size}.'
    )
    return np.inf

  # --- Call the fast Numba function for the core calculation ---
  # The first call will have a slight compilation overhead. Subsequent calls are free.
  sum_set_size, prod_set_size = _calculate_sum_prod_numba(construction, p)

  # The score is the maximum of the two sizes. Lower is better.
  score = float(max(sum_set_size, prod_set_size))

  return score

**Prompts used**

**First version**:

Problem Statement:

The finite field sum-product problem asks for sets that have both a small sum set and a small product set. Your goal is to construct a set X within the finite field F_p for a large prime p (which will always around 1 billion, and it will be congruent to 1 mod 4), such that the set is simultaneously "structured" under both addition and multiplication.

A set X subseteq F_p has a sum set X+X = (x_1 + x_2 (mod p) | x_1, x_2 in X) and a product set X * X = (x_1 * x_2 (mod p) | x_1, x_2 in X). The sum-product conjecture states that for any set X, at least one of these two sets must be large. Your task is to find a set X that challenges this conjecture by making max(|X+X|, |X*X|) as small as possible.

Your Task:

You must write a Python function, search_for_best_construction, that takes a prime p and returns a proposed set X.

Input:
p: A large prime number.

Function Constraints:
The size of the set X you return must be exactly floor(sqrt(p)).
All elements in X must be unique and belong to [0, 1, ..., p-1].

Output:

A 1D NumPy array of shape (k,) where k = floor(sqrt(p)), representing the elements of your set X. The elements must be of type np.int64.

How Your Function is Used:

Your function directly produces the set X. An external function will then compute the sizes of its sum set (|X+X|) and product set (|X*X|) and evaluate your construction based on these sizes.

Your Goal and Evaluation:

Your goal is to define the set X in such a way that max(|X+X|, |X*X|) is minimized. Your function will be evaluated based on the average value of this maximum size over several large primes. A lower size is better. If your function returns a set that violates the constraints (e.g., wrong size, duplicate elements), it will receive a very large penalty.

Hints and Benchmarks:
A random set X of size k is expected to have |X+X| and |XX| both on the order of k^2, which is very large.
An arithmetic progression (e.g., [1, 2, ..., k]) has a very small sum set (|X+X| = 2k-1) but a very large product set.
A geometric progression (e.g., [1, g, g^2, ..., g^(k-1)]) has a very small product set (|XX| = 2k-1) but a very large sum set.
The challenge is to find a construction that is not a simple AP or GP, but has some other algebraic structure that keeps both sets small. Any construction that achieves max(|X+X|, |X*X|) < |X|^1.5 would be interesting.

The normalized_score you are seeing describes the exponent of the previous constructions, i.e. the number a such that |X|^a = max(|X+X|, |X*X|). This is upper bounded by about 2 (since |X| will always be floor(sqrt(p))), and lower is better.

Good luck!

**Second version**:

Problem Statement:

The finite field sum-product problem asks for sets that have both a small sum set and a small product set. Your goal is to construct a set X within the finite field F_p for a large prime p (which will always around 1 billion, and it will be congruent to 1 mod 4), such that the set is simultaneously "structured" under both addition and multiplication.

A set X subseteq F_p has a sum set X+X = (x_1 + x_2 (mod p) | x_1, x_2 in X) and a product set X * X = (x_1 * x_2 (mod p) | x_1, x_2 in X). The sum-product conjecture states that for any set X, at least one of these two sets must be large. Your task is to find a set X that challenges this conjecture by making max(|X+X|, |X*X|) as small as possible.

Your Task:

You must write a Python function, search_for_best_construction, that takes a prime p and returns a proposed set X.

Input:
p: A large prime number.

Function Constraints:
The size of the set X you return must be exactly floor(sqrt(p)).
All elements in X must be unique and belong to [0, 1, ..., p-1].

Output:

A 1D NumPy array of shape (k,) where k = floor(sqrt(p)), representing the elements of your set X. The elements must be of type np.int64.

How Your Function is Used:

Your function directly produces the set X. An external function will then compute the sizes of its sum set (|X+X|) and product set (|X*X|) and evaluate your construction based on these sizes.

Your Goal and Evaluation:

Your goal is to define the set X in such a way that max(|X+X|, |X*X|) is minimized. Your function will be evaluated based on the average value of this maximum size over several large primes. A lower size is better. If your function returns a set that violates the constraints (e.g., wrong size, duplicate elements), it will receive a very large penalty.

Hints and Benchmarks:
A random set X of size k is expected to have |X+X| and |XX| both on the order of k^2, which is very large.
An arithmetic progression (e.g., [1, 2, ..., k]) has a very small sum set (|X+X| = 2k-1) but a very large product set.
A geometric progression (e.g., [1, g, g^2, ..., g^(k-1)]) has a very small product set (|XX| = 2k-1) but a very large sum set.
The challenge is to find a construction that is not a simple AP or GP, but has some other algebraic structure that keeps both sets small. You can get a normalized score of around 1.5 easily by intersecting an AP with a GP in a clever way. This is NOT the type of construction you should go for. It turn out there is a way to get |X+X| and |X*X| to both be below |X|^1.4-ish. Your goal is to rediscover this |X|^1.4-ish construction. Once you reach an exponent of 1.4 you are on your own. How low can you push this exponent?

The normalized_score you are seeing describes the exponent of the previous constructions, i.e. the number a such that |X|^a = max(|X+X|, |X*X|). This is upper bounded by about 2 (since |X| will always be floor(sqrt(p))), and lower is better. The best known construction achieves a normalized_score of around 1.5 by using smooth integers. Your goal is to reproduce this result, and if you can, try to beat it.

Good luck!

In [ ]:
#@title Initial program used

"""AlphaEvolve experiment for the finite field sum-product problem."""
import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from collections.abc import Callable, Mapping
from typing import Any, List, Tuple
import scipy.linalg as la
import collections
import copy
import math
import numba
from scipy.optimize import milp, LinearConstraint, Bounds
from itertools import product

njit = numba.njit


def search_for_best_construction(p: int) -> np.ndarray:
  """Constructs a set X in F_p of size floor(sqrt(p)).

  The goal is to minimize max(|X+X|, |X*X|).
  """
  # The size of the set is fixed to floor(sqrt(p)).
  size = int(math.sqrt(p))

  # A simple baseline construction: an arithmetic progression.
  # This construction has a very small sum set, but a large product set.
  # The challenge is to find a set where BOTH are small.
  x = np.arange(size, dtype=np.int64)

  return x